[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main//buscador/busqueda_inteligente_openai.ipynb)

# ===============================================================
#🧠 ASISTENTE INTELIGENTE DE DOCUMENTOS EN ONEDRIVE + OPENAI
# ===============================================================

# Descripción:
Este script conecta tu cuenta de OneDrive, analiza los documentos
almacenados en una carpeta específica y los relaciona con una tabla
de etiquetas para responder preguntas en lenguaje natural, por ejemplo:
  - "¿Qué documentos están relacionados con la contabilidad?"
  - "Muéstrame los archivos que traten sobre políticas internas o contratos."
  - "¿Qué informes financieros mencionan resultados del último trimestre?"

# Resultado:
El sistema actúa como un “asistente documental” capaz de comprender
preguntas complejas y sugerir los archivos más relacionados, combinando
etiquetas, contenido textual y lenguaje natural.

# Requisitos previos:
- Acceso a una cuenta Microsoft (OneDrive) con los permisos correctos.
- Archivo CSV con la tabla de etiquetas (`etiquetas_onedrive.csv`).
- API key válida de OpenAI.

Autor: Patricia Puga y Marco Mendieta
Proyecto: Búsqueda inteligente de documentos OneDrive + OpenAI


[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#1. Dependencias

In [1]:
!pip install -q PyPDF2 python-docx openpyxl python-pptx xlrd transformers office365-rest-python-client msal requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 8.0 MB/s eta 0:00:00


#2. Importaciones y configuración

In [8]:
import os
import io
import re
import json
from pathlib import Path
from PyPDF2 import PdfReader
import docx
import openpyxl
from pptx import Presentation
import xlrd
from transformers import pipeline
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import os
import requests
import msal
import csv
import getpass
from openai import OpenAI
from transformers import pipeline
import time
import openai
import requests
import pandas as pd

# 3. Autenticaciones




API KEY

In [3]:
api_key = getpass.getpass("Introduce tu OpenAI API Key: ")
client = OpenAI(api_key=api_key)
token = getpass.getpass("Introduce tu GitHub token: ")

Introduce tu OpenAI API Key: ··········
Introduce tu GitHub token: ··········


Conección con OneDrive

In [4]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"  # tu nuevo Client ID
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"

In [5]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()

To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code YZQEEFEB to authenticate.


#4. Leer archivos de OneDrive

In [6]:
def leer_pdf(contenido, limite_palabras=None):
    texto = ""
    reader = PdfReader(io.BytesIO(contenido))
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            texto += page_text + "\n"
    # Aplicar límite de palabras al final
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_docx(contenido, limite_palabras=None):
    texto = ""
    doc = docx.Document(io.BytesIO(contenido))
    # Párrafos
    for p in doc.paragraphs:
        if p.text.strip():
            texto += p.text + "\n"
    # Tablas
    for table in doc.tables:
        for row in table.rows:
            row_text = " ".join([cell.text for cell in row.cells if cell.text.strip()])
            if row_text:
                texto += row_text + "\n"
    # Aplicar límite al final
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_excel(contenido, limite_palabras=None):
    texto = ""
    wb = openpyxl.load_workbook(io.BytesIO(contenido), data_only=True, read_only=True)
    for sheet in wb.worksheets:
        for row in sheet.iter_rows(values_only=True):
            row_text = " ".join([str(cell) for cell in row if cell])
            if row_text:
                texto += row_text + "\n"
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_xls(contenido, limite_palabras=None):
    texto = ""
    temp_file = "temp.xls"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    wb = xlrd.open_workbook(temp_file)
    for sheet in wb.sheets():
        for row_idx in range(sheet.nrows):
            row = sheet.row_values(row_idx)
            row_text = " ".join([str(cell) for cell in row if cell])
            if row_text:
                texto += row_text + "\n"
    os.remove(temp_file)
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_pptx(contenido, limite_palabras=None):
    texto = ""
    temp_file = "temp.pptx"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    prs = Presentation(temp_file)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                texto += shape.text + "\n"
    os.remove(temp_file)
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_txt_csv(contenido, limite_palabras=None):
    texto = contenido.decode("utf-8", errors="ignore")
    if limite_palabras:
        texto = " ".join(texto.split()[:limite_palabras])
    return texto

def leer_archivo(nombre, contenido, limite_palabras=None):
    ext = nombre.split(".")[-1].lower()
    if ext == "pdf":
        return leer_pdf(contenido, limite_palabras)
    elif ext == "docx":
        return leer_docx(contenido, limite_palabras)
    elif ext == "xlsx":
        return leer_excel(contenido, limite_palabras)
    elif ext == "xls":
        return leer_xls(contenido, limite_palabras)
    elif ext == "pptx":
        return leer_pptx(contenido, limite_palabras)
    elif ext in {"txt", "csv"}:
        return leer_txt_csv(contenido, limite_palabras)
    else:
        return ""

In [9]:
# Extensiones que te interesan
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}

# Filtrar solo archivos válidos dentro del JSON obtenido
archivos_permitidos = [
    {
        "nombre": item["name"],
        "url_descarga": item["@microsoft.graph.downloadUrl"],
        "tamano_kb": round(item["size"] / 1024, 2)
    }
    for item in data.get("value", [])
    if item["name"].split(".")[-1].lower() in ext_permitidas
]

# Mostrar los archivos encontrados
if not archivos_permitidos:
    print("⚠️ No se encontraron archivos con las extensiones permitidas.")
else:
    print("📂 Archivos encontrados en OneDrive:")
    for f in archivos_permitidos:
        print(f" - {f['nombre']} ({f['tamano_kb']} KB)")

# (Opcional) crear un DataFrame con la lista de archivos
df_archivos = pd.DataFrame(archivos_permitidos)
print("\n✅ Total de archivos válidos encontrados:", len(df_archivos))
df_archivos.head()


📂 Archivos encontrados en OneDrive:
 - 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf (689.21 KB)
 - 2016-admitidos_Segundo ciclo- Cursos monográficos.xls (134.5 KB)
 - 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx (35.8 KB)
 - 660d1bfb7c43622a597a4000_Non-disclosure agreement nda template contract.pdf (20.88 KB)
 - Acuerdo_no_Divulgacion_Unilateral_UE.pdf (168.59 KB)
 - Acuerdo-de-Confidencialidad-OEPM.pdf (334.54 KB)
 - AnaLiliana_SuarezHerrera_WelmarFernandoRinconTorres_2018_Anexo1.pptx (1173.64 KB)
 - Analisis de interpretacion de estados financieros.pptx (519.51 KB)
 - Análisis Financiero del Proyecto.xls (609.0 KB)
 - ANALISIS FINANCIERO.xls (97.5 KB)
 - biblioteca2-csv.xls (328.5 KB)
 - contrato_alquiler_opcion_compra.pdf (242.73 KB)
 - ContratoAlquiler_reducido.pdf (135.38 KB)
 - contrato-de-confidencialidad-freelancer-plantilla-gratis-word.docx (22.63 KB)
 - Formulario_EX00_I28202503542693.pdf (425.18 KB)
 - Formulario_EX00_I

,nombre,url_descarga,tamano_kb
0,11_07_2019_modelo_orientativo_de_contrato_de_a...,https://my.microsoftpersonalcontent.com/person...,689.21
1,2016-admitidos_Segundo ciclo- Cursos monográfi...,https://my.microsoftpersonalcontent.com/person...,134.50
2,2023_05-Modelo_Documento_reserva_inmueble_en_a...,https://my.microsoftpersonalcontent.com/person...,35.80
3,660d1bfb7c43622a597a4000_Non-disclosure agreem...,https://my.microsoftpersonalcontent.com/person...,20.88
4,Acuerdo_no_Divulgacion_Unilateral_UE.pdf,https://my.microsoftpersonalcontent.com/person...,168.59


#5. Cargar la tabla de documentos y etiquetas

In [10]:
df_etiquetas = pd.read_csv("https://raw.githubusercontent.com/pugapatricia/gestion-documentaria-para-pymes/refs/heads/main/etiquetado/etiquetas_onedrive.csv", encoding="utf-8-sig")
df_etiquetas.head()

,Archivo,Etiquetas
0,11_07_2019_modelo_orientativo_de_contrato_de_a...,"registro, propiedad, arrendador, indicadores, ..."
1,2016-admitidos_Segundo ciclo- Cursos monográfi...,"historia, confidencialidad, arte, medioambient..."
2,2023_05-Modelo_Documento_reserva_inmueble_en_a...,"seguro, seguridad, obligaciones, solicitud, ju..."
3,660d1bfb7c43622a597a4000_Non-disclosure agreem...,"solicitud, propiedadintelectual, balance, incu..."
4,Acuerdo_no_Divulgacion_Unilateral_UE.pdf,"cesión, medioambiente, arrendatario, registro,..."


In [11]:
df_etiquetas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Archivo    25 non-null     object
 1   Etiquetas  25 non-null     object
dtypes: object(2)
memory usage: 532.0+ bytes


#6. Leer documentos

In [13]:
import requests

# Crear diccionario para almacenar texto extraído
textos_archivos = {}

# Puedes limitar la cantidad de archivos a leer (por tiempo)
archivos_a_procesar = df_archivos.head(10).to_dict("records")  # por ejemplo, los 5 primeros

for item in archivos_a_procesar:
    nombre = item["nombre"]
    url = item["url_descarga"]

    print(f"📖 Leyendo contenido de: {nombre} ...")
    try:
        # Descargar contenido binario
        file_bytes = requests.get(url).content

        # Leer según tipo de archivo (usando tus funciones)
        texto = leer_archivo(nombre, file_bytes, limite_palabras=1000)

        # Guardar texto limpio
        textos_archivos[nombre] = texto

        print(f"✅ {nombre}: {len(texto.split())} palabras extraídas")

    except Exception as e:
        print(f"⚠️ Error al leer {nombre}: {e}")

print("\n🗂️ Total de archivos leídos:", len(textos_archivos))


📖 Leyendo contenido de: 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf ...
✅ 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf: 1000 palabras extraídas
📖 Leyendo contenido de: 2016-admitidos_Segundo ciclo- Cursos monográficos.xls ...
✅ 2016-admitidos_Segundo ciclo- Cursos monográficos.xls: 1000 palabras extraídas
📖 Leyendo contenido de: 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx ...
✅ 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx: 1000 palabras extraídas
📖 Leyendo contenido de: 660d1bfb7c43622a597a4000_Non-disclosure agreement nda template contract.pdf ...
✅ 660d1bfb7c43622a597a4000_Non-disclosure agreement nda template contract.pdf: 609 palabras extraídas
📖 Leyendo contenido de: Acuerdo_no_Divulgacion_Unilateral_UE.pdf ...
✅ Acuerdo_no_Divulgacion_Unilateral_UE.pdf: 1000 palabras extraídas
📖 Leyendo contenido de: Acuerdo-de-Confidencialidad-OEPM.pdf ...
✅ Acuerdo-de-Confidencialidad

#7. Función para encontrar documentos relacionados

In [22]:
def buscar_documentos(consulta, df_etiquetas, textos_archivos, client, max_resultados=5, max_tokens_preview=1000):
    if not consulta.strip():
        print("⚠️ La consulta está vacía. Por favor escribe algo.")
        return pd.DataFrame(columns=["Archivo", "Puntuación", "Etiquetas"])

    print(f"\n🔍 Buscando documentos relacionados con: '{consulta}' ...")

    resultados = []

    for nombre, texto in textos_archivos.items():
        etiquetas_doc = ""
        if "Archivo" in df_etiquetas.columns and "Etiquetas" in df_etiquetas.columns:
            match = df_etiquetas[df_etiquetas["Archivo"].str.lower() == nombre.lower()]
            if not match.empty:
                etiquetas_doc = str(match["Etiquetas"].iloc[0])

        preview_text = texto[:max_tokens_preview]

        prompt = f"""
        Consulta del usuario: {consulta}

        Documento: {nombre}
        Etiquetas: {etiquetas_doc}

        Contenido del documento (extracto):
        {preview_text}

        Devuelve únicamente una puntuación del 0 al 10 indicando qué tan relevante es este documento respecto a la consulta.
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Eres un analista que evalúa la relevancia de documentos."},
                    {"role": "user", "content": prompt}
                ]
            )

            respuesta = response.choices[0].message.content.strip()
            puntuacion = next((float(s) for s in respuesta.split() if s.replace('.', '', 1).isdigit()), 0)
            puntuacion = max(0, min(10, puntuacion))

        except Exception as e:
            print(f"⚠️ Error al analizar {nombre}: {e}")
            puntuacion = 0

        resultados.append({"Archivo": nombre, "Puntuación": puntuacion, "Etiquetas": etiquetas_doc})

    df_resultados = pd.DataFrame(resultados)
    df_resultados = df_resultados[df_resultados["Puntuación"] >= 1]
    df_resultados.sort_values("Puntuación", ascending=False, inplace=True)
    df_resultados.reset_index(drop=True, inplace=True)

    if df_resultados.empty:
        print("❌ No se encontraron documentos relevantes con puntuación >= 1.")
        return df_resultados

    print("\nDocumentos más relevantes:")
    display(df_resultados.head(max_resultados))

    return df_resultados.head(max_resultados)


#8. Ejecutar la búsqueda

In [23]:
while True:
    consulta = input("\n💬 Escribe tu consulta (o escribe 'salir' para terminar): ").strip()

    if consulta.lower() in ["salir", "exit"]:
        print("👋 Saliendo del buscador inteligente. ¡Hasta luego!")
        break

    resultados = buscar_documentos_simple(consulta, df_etiquetas)

    if resultados is not None:
        print("\n📊 Resultados mostrados arriba.\n")



💬 Escribe tu consulta (o escribe 'salir' para terminar): huuzz

🔍 Buscando documentos relacionados con: 'huuzz' ...

❌ No hay documentos relacionados con 'huuzz'

💬 Escribe tu consulta (o escribe 'salir' para terminar): contratos

🔍 Buscando documentos relacionados con: 'contratos' ...

❌ No hay documentos relacionados con 'contratos'

💬 Escribe tu consulta (o escribe 'salir' para terminar): documentos

🔍 Buscando documentos relacionados con: 'documentos' ...

❌ No hay documentos relacionados con 'documentos'

💬 Escribe tu consulta (o escribe 'salir' para terminar): exit
👋 Saliendo del buscador inteligente. ¡Hasta luego!
